<a href="https://colab.research.google.com/github/Albert840529/Composite_CT_Thickness_calculation/blob/main/CT_Image_Thickness_Calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connect to googel drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Test loading the image and see how it looks like. After Ostu method, got the threshold value.

In [ ]:
# /content/drive/MyDrive/Colab Notebooks/test/IPP15_10.jpg
import cv2
from google.colab.patches import cv2_imshow

# Load the image (Please use your own route for the image you want to analyze)
img = cv2.imread("/content/drive/MyDrive/Colab Notebooks/test/Ipp_15_10_crop.jpg", 0)

# Apply Otsu's method to find the threshold value
ret, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Display the binary image
cv2_imshow(thresh)

# Test the code for calculating the aggregate gypsum thickness on one image

In [ ]:
import numpy as np

def one_image_thickness(height, width, thresh):
    avg_white_pixels = []
    for w in range(width):
        white_pixels = []
        for h in range(height):
            if thresh[h,w] == 255:
                white_pixels.append(h)
        if len(white_pixels) > 0:
            white_pixels = np.array(white_pixels)
            diff = np.diff(white_pixels)
            continuous = np.where(diff > 1)[0]
            if len(continuous) > 0:
                segments = np.split(white_pixels, continuous + 1)
                lengths = [len(segment) for segment in segments]
                avg_white_pixels.append(sum(lengths) / len(lengths))
            else:
                avg_white_pixels.append(len(white_pixels))
        else:
            avg_white_pixels.append(0)
    return avg_white_pixels

height, width = thresh.shape
avg_white_pixels = one_image_thickness(height, width, thresh)

# Display the average continuous white pixels at each width position
# print(avg_white_pixels)
# print(np.mean(avg_white_pixels)*7.5, np.std(avg_white_pixels))

# Code for read all image and calculate gypsum thickness for all image, output into csv file. 

In [ ]:
import cv2
import numpy as np
import os
import csv

def calculate_thickness(input_dirs, pixel_size=7.5):
    i = 1
    for input_dir in input_dirs:
        print(f'Runnning {i} of {len(input_dirs)} folders')
        print(f'Now working on {input_dir}...............')
         # Define the output directory as the same as input directory
        output_dir = input_dir

        # Define the output CSV file name as the same as the input directory name
        output_file = os.path.join(output_dir, os.path.basename(input_dir) + '.csv')
        # Initialize the output CSV file
        with open(output_file, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['Image Name', 'Average White Pixels', 'Standard Deviation', 'Average Thickness(µm)', 'Standard Deviation of Thickness', 'UTS (MPa)',
                             'Elongation at break (%)', "Young's Modulus (GPa)" ])

        # Loop over all the images in the input directory
        for filename in os.listdir(input_dir):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                # Load the binary image
                img = cv2.imread(os.path.join(input_dir, filename), 0)
                ret, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

                # Scan each width position and compute the average continuous white pixels
                height, width = thresh.shape
                avg_white_pixels = one_image_thickness(height, width, thresh)

                # Compute the average and standard deviation of the average continuous white pixels
                avg = np.mean(avg_white_pixels)
                std = np.std(avg_white_pixels)
                avg_thickness = avg * pixel_size
                std_thickness = std * pixel_size
                folder_name = os.path.basename(os.path.normpath(input_dir))
                uts_value = folder_name.split('_')[-3]
                elongation = float(folder_name.split('_')[-1])
                ym = float(folder_name.split('_')[-2])

                # Append the data to the output CSV file
                with open(output_file, 'a', newline='') as csvfile:
                    writer = csv.writer(csvfile)
                    writer.writerow([filename, avg, std, avg_thickness, std_thickness, uts_value, elongation, ym])

        i += 1


In [ ]:
def get_folders(path):
    folders = []
    for root, dirs, files in os.walk(path):
        for dir in dirs:
            print(dir)
            folders.append(os.path.join(root, dir))
    return folders

# Route for all images you want to analyze 
folders = get_folders('/content/drive/MyDrive/Colab Notebooks/MatSE597/Project/test')

In [ ]:
# In each subfolder, you will have the analyzed CSV file with the calculated thickness
calculate_thickness(folders)

Runnning 1 of 2 folders
Now working on /content/drive/MyDrive/Colab Notebooks/MatSE597/Project/test/IPP_5%_6_0.88_2.22_1.45...............
Runnning 2 of 2 folders
Now working on /content/drive/MyDrive/Colab Notebooks/MatSE597/Project/test/R_10_IPP_20%_3_4.46_2.01_1.33...............
